# <font color='orange'>Notebook de requête API</font>

LIVRABLE : Un notebook de requête API, qui récupère 50 questions par requête via le wrapper StackAPI ou bien via Stack Exchange API, et qui stocke les principales caractéristiques de ces questions dans un DataFrame.

CONSIGNE : Il existe également une API qui permet de récupérer les informations des questions de StackOverflow. Je souhaiterais vérifier la pertinence de cette API et la qualité de la documentation pour l’utiliser.

Pourrais-tu donc, juste pour tester cette API, réaliser dans un notebook une requête de 50 questions sur une période définie, contenant le tag “python” et qui ont un score > 50 (votes), récupérer les données principales de la question (date, titre, tags, score) dans un DataFrame et les afficher ?

Tu pourras utiliser le wrapper Python StackAPI ou bien directement la “Stack Exchange API”.

Ton retour sur son utilisation me sera précieux, pour éventuellement l’améliorer.

Pense également à bien respecter les grands principes de gestion de données personnelles du RGPD, en particulier en ne récupérant que les données nécessaires pour le projet et en gardant aucune information permettant d’identifier les personnes qui sont les auteurs des questions.

In [4]:
# Installation de StackAPI.
# pip install stackapi

Récupération des questions de Stack Overflow en utilisant la bibliothèque StackAPI

In [6]:
# Import des bibliothèques.
import random
from stackapi import StackAPI
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [7]:
try:
    # Définition des paramètres.
    nb_questions_total = 500  # Nombre total de questions à récupérer.
    nb_questions_sample = 50  # Nombre de questions à échantillonner aléatoirement.
    tags_liste = ['python']
    seuil_min_score = 50

    # Choix de la période pour la variable date.
    nb_n_old = 13  # Choix de l'ancienneté des données en années.
    date_max = datetime.now()  # Date fin de période.
    date_min = datetime.now() - relativedelta(years=nb_n_old)  # Calcul de la date début de période.
    # Conversion des dates en timestamps.
    date_max_timestamp = int(date_max.timestamp())
    date_min_timestamp = int(date_min.timestamp())
    
    # Configuration de la requête API.
    SITE = StackAPI('stackoverflow')
    SITE.page_size = min(nb_questions_total, 100)  # Page size maximum 100.
    SITE.max_pages = nb_questions_total // SITE.page_size  # Calcule le nombre de pages.

    # Récupération des questions triées par votes.
    questions = SITE.fetch('questions', 
                           tagged=tags_liste, 
                           sort='votes'  # Tri des questions par nombre de votes desc.
                          )

    # Filtrage des données par score et date.
    data = []
    items = questions.get('items', [])
    
    # Filtrer par date
    filtered_items = [item for item in items 
                      if date_min_timestamp <= item.get('creation_date', 0) <= date_max_timestamp]
    
    # Filtrer par score minimum
    filtered_items = [item for item in filtered_items if item.get('score', 0) > seuil_min_score]
    
    if len(filtered_items) >= nb_questions_sample:
        sampled_items = random.sample(filtered_items, nb_questions_sample)
    else:
        print(f"Nombre insuffisant de questions avec un score > {seuil_min_score}. Total trouvé: {len(filtered_items)}")
        sampled_items = filtered_items

    for item in sampled_items:
        question_data = {
            'creation_date': pd.to_datetime(item.get('creation_date', 0), unit='s'),
            'title': item.get('title', 'N/A'),
            'tags': ', '.join(item.get('tags', [])) if item.get('tags') else 'N/A',
            'score': item.get('score', 0)
        }
        data.append(question_data)

    # Création du DataFrame.
    df = pd.DataFrame(data)

except Exception as e:
    print(f"An error occurred: {e}")

In [8]:
df

,title,creation_date,tags,score
0,"When to use cla(), clf() or close() for cleari...",2011-11-21 14:38:03,"python, matplotlib, plot",761
1,Why does Python code run faster in a function?,2012-06-28 09:18:34,"python, performance, profiling, benchmarking, ...",934
2,Removing Conda environment,2018-03-06 09:49:50,"python, jupyter, conda, environment",901
3,Why is python setup.py saying invalid command ...,2016-01-15 20:19:56,"python, travis-ci, setup.py, pypi",927
4,How do I count the NaN values in a column in p...,2014-10-08 21:00:19,"python, pandas, dataframe",814
5,"Difference between map, applymap and apply met...",2013-11-05 20:20:14,"python, pandas, dataframe, vectorization",741
6,Search a list of dictionaries in Python,2011-12-28 08:25:54,"python, search, dictionary",804
7,How to check if a dictionary is empty?,2014-04-20 01:29:19,"python, dictionary",720
8,Getting today&#39;s date in YYYY-MM-DD in Python?,2015-09-09 23:20:25,"python, datetime, python-datetime",1012
9,How to disable Python warnings?,2013-01-22 16:26:49,"python, suppress-warnings",931


Il y a très peu de questions qui cumulent plusieurs conditions (içi tags = "python" et nombre de vote > 50). Il peut être compliqué de récupérer un grand nombre de résultats si l'on souhaite des données très spécifiques. 

In [ ]:
STOP

Je vais commencer par récupérer les données d'entraînement en utilisant la requête SQL fournie. Ensuite, nous pourrons commencer à construire le modèle de suggestion de tags. En attendant, je vais également examiner l'API Stack Exchange pour récupérer les informations des questions comme demandé.

Avant de commencer à développer l'algorithme de suggestion de tags, je vais d'abord vérifier la pertinence de l'API Stack Exchange et la qualité de sa documentation en récupérant les données des questions de Stack Overflow.

Pour ce faire, nous allons utiliser la bibliothèque StackAPI en Python pour effectuer une requête de 50 questions sur une période définie, contenant le tag "python" et ayant un score supérieur à 50 (votes). Ensuite, nous afficherons les données principales de chaque question dans un DataFrame.